## Simple benchmark using XGBoost

In [1]:
import os
import sys
import pandas as pd

In [2]:
sys.path.insert(0, os.path.abspath('/home/jupyter/kaggle/predict_future_sales/src/'))

In [3]:
%load_ext autoreload
%autoreload 2

In [38]:
import common.com_util as util
import munging.process_data as process_data
import config.constants as constants
import cv.cv_util as cv

In [24]:
logger = util.get_logger('sub_2')

In [ ]:
logger = util.get_logger

In [8]:
train_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/train_all_merged.feather')
test_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/test_all_merged.feather')

train_features = ['shop_id', 'item_id', 'date_block_num', 'item_category_id', 'item_cnt_month']
train_df = train_df[train_features]

test_features = ['ID', 'shop_id', 'item_id', 'date_block_num', 'item_category_id']
test_df = test_df[test_features]

In [21]:
train_df.head()

,shop_id,item_id,date_block_num,item_category_id,item_cnt_month
0,0,19,0,40,0.0
1,0,27,0,19,0.0
2,0,28,0,30,0.0
3,0,29,0,23,0.0
4,0,32,0,40,6.0


In [22]:
test_df.head()

,ID,shop_id,item_id,date_block_num,item_category_id
0,0,5,5037,34,19
1,1,5,5320,34,55
2,2,5,5233,34,19
3,3,5,5232,34,23
4,4,5,5268,34,20


In [25]:
train_df.shape

(10913850, 5)

In [26]:
test_df.shape

(214200, 5)

In [29]:
TARGET = 'item_cnt_month'
ID = 'ID'

In [30]:
combined_df = pd.concat([train_df.drop(TARGET, axis=1), test_df.drop(ID, axis=1)])
logger.info(f'Shape of the combined DF {combined_df.shape}')

[INFO]2020-07-30 04:42:57,622:sub_2:Shape of the combined DF (11128050, 4)


In [36]:
train_index = train_df.shape[0]
train_Y = train_df[TARGET]

train_X = combined_df[:train_index]
test_X = combined_df[train_index:]

In [37]:
logger.info(f"train_X : {train_X.shape}")
logger.info(f"test_X : {test_X.shape}")
logger.info(f"train_Y : {train_Y.shape}")

[INFO]2020-07-30 04:45:34,223:sub_2:train_X : (10913850, 4)
[INFO]2020-07-30 04:45:34,224:sub_2:test_X : (214200, 4)
[INFO]2020-07-30 04:45:34,225:sub_2:train_Y : (10913850,)


In [39]:
combined_df.date_block_num.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34])

In [40]:
training_months = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
holdout_months = [33]

In [45]:
training, validation = cv.get_data_splits_by_date_block(logger, combined_df, 
                                                        train_months=training_months, 
                                                        validation_months=holdout_months)

[INFO]2020-07-30 05:08:01,696:sub_2:Splitting the data into train and holdout based on months...
[INFO]2020-07-30 05:08:01,697:sub_2:Training months [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
[INFO]2020-07-30 05:08:01,698:sub_2:Validation months [33]
[INFO]2020-07-30 05:08:02,609:sub_2:Shape of the training data (10675678, 4) 
[INFO]2020-07-30 05:08:02,610:sub_2:Shape of the validation data (238172, 4)
